<a href="https://colab.research.google.com/github/pulkit48/Conversational-Chatbot-with-Query-Expansion-and-Domain-Predcition/blob/main/colab/Query_Expansion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install torch
import torch

In [ ]:
torch.cuda.empty_cache()

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

2024-05-28 14:17:17.753582: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-28 14:17:17.753717: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-28 14:17:17.926047: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [ ]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
new_model = "Llama-2-7b-chat-finetune"

import pandas as pd
#https://drive.google.com/file/d/1VdCTB65pUVaIcW4D52TE83cVuDvIfRCY/view?usp=sharing
file_id="1VdCTB65pUVaIcW4D52TE83cVuDvIfRCY"
download_link = f"https://drive.google.com/uc?id={file_id}"

df = pd.read_csv(download_link)

In [ ]:
df.head()

,Unnamed: 0,Context,Question,Rewrite,Answer,Answer_URL,Conversation_no,Turn_no,Conversation_source
0,0,[],What can you tell me about Gary Cherone?,What can you tell me about Gary Cherone?,Gary Francis Caine Cherone is an American rock...,https://en.wikipedia.org/wiki/Van_Halen,1,1,quac
1,1,"['What can you tell me about Gary Cherone?', '...",Did Gary sing well?,Did Gary Cherone sing well?,"Yes, Gary Cherone is also known for his work a...",https://en.wikipedia.org/wiki/Van_Halen,1,2,quac
2,2,"['What can you tell me about Gary Cherone?', '...",What significant fact can you tell me about Ga...,What significant fact can you tell me about Ga...,I like that Gary Cherone remained in contact a...,https://en.wikipedia.org/wiki/Van_Halen,1,3,quac
3,3,"['What can you tell me about Gary Cherone?', '...",What did Cherone do after Van Halen?,What did Gary Cherone do after Van Halen?,"After his departure from Van Halen, Gary Chero...",https://en.wikipedia.org/wiki/Van_Halen,1,4,quac
4,4,"['What can you tell me about Gary Cherone?', '...",Did they release any albums during that time f...,Did they release any albums after Gary Cherone...,"After Gary Cherone, Eddie Van Halen recovered ...",https://en.wikipedia.org/wiki/Van_Halen,1,5,quac


In [ ]:
df=df.drop(columns=["Unnamed: 0","Context","Answer_URL","Turn_no","Conversation_source"])

In [ ]:
df

,Question,Rewrite,Answer,Conversation_no
0,What can you tell me about Gary Cherone?,What can you tell me about Gary Cherone?,Gary Francis Caine Cherone is an American rock...,1
1,Did Gary sing well?,Did Gary Cherone sing well?,"Yes, Gary Cherone is also known for his work a...",1
2,What significant fact can you tell me about Ga...,What significant fact can you tell me about Ga...,I like that Gary Cherone remained in contact a...,1
3,What did Cherone do after Van Halen?,What did Gary Cherone do after Van Halen?,"After his departure from Van Halen, Gary Chero...",1
4,Did they release any albums during that time f...,Did they release any albums after Gary Cherone...,"After Gary Cherone, Eddie Van Halen recovered ...",1
...,...,...,...,...
63496,Where does Call of the Dead take place,Where does Call of the Dead take place,It takes place in a desolate area of the Siber...,10823
63497,What is the theme,What is Call ForThe Dead's theme,The players are once again are tasked with sur...,10823
63498,What is the genre,What is the genre of Call For The Dead,"The genre is crime, spy novel.",10823
63499,Who plays the main role,Who plays the main role in Call For The Dead,Simon Russell Beale plays the main role ij Cal...,10823


In [ ]:
list1=[]
dict1={}
for index,rows in df.iterrows():
    id=rows["Conversation_no"]
    question=rows["Question"]
    rewrite=rows["Rewrite"]
    answer=rows['Answer']

    if id not in dict1:
        dict1[id]="<s>"
        temp_str1=dict1[id]+f'''
        <INST>Question:{question}</INST>
        Updated Question:{rewrite}
        Answer:{answer}\n</s>'''
        list1.append(temp_str1)

        temp_str2=f'''
        Q:{question}
        UQ:{rewrite}
        A:{answer}'''
        dict1[id]=dict1[id]+temp_str2

    else:
        temp_str1=dict1[id]+f'''

        <INST>Question:{question}</INST>
        Updated Question:{rewrite}
        Answer:{answer}\n</s>'''
        list1.append(temp_str1)

        temp_str2=f'''
        Q:{question}
        UQ:{rewrite}
        A:{answer}'''
        dict1[id]=dict1[id]+temp_str2



print(list1[4])

<s>
        Q:What can you tell me about Gary Cherone?
        UQ:What can you tell me about Gary Cherone?
        A:Gary Francis Caine Cherone is an American rock singer and songwriter, known for his work as the lead vocalist of Extreme and for his short stint for Van Halen.
        Q:Did Gary sing well?
        UQ:Did Gary Cherone sing well?
        A:Yes, Gary Cherone is also known for his work as the lead vocalist of the Boston rock group Extreme.
        Q:What significant fact can you tell me about Gary that you liked?
        UQ:What significant fact can you tell me about Gary Cherone that you liked?
        A:I like that Gary Cherone remained in contact and on good terms with Van Halen.
        Q:What did Cherone do after Van Halen?
        UQ:What did Gary Cherone do after Van Halen?
        A:After his departure from Van Halen, Gary Cherone returned to Boston and put together a new project, Tribe of Judah.
    
        <INST>Question:Did they release any albums during that ti

In [ ]:
df_list=list1[0:1000]
temp=[]
for i in df_list:
  temp.append({"text":i})

In [ ]:
temp[5]

{'text': "<s>\n        Q:What can you tell me about Gary Cherone?\n        UQ:What can you tell me about Gary Cherone?\n        A:Gary Francis Caine Cherone is an American rock singer and songwriter, known for his work as the lead vocalist of Extreme and for his short stint for Van Halen.\n        Q:Did Gary sing well?\n        UQ:Did Gary Cherone sing well?\n        A:Yes, Gary Cherone is also known for his work as the lead vocalist of the Boston rock group Extreme.\n        Q:What significant fact can you tell me about Gary that you liked?\n        UQ:What significant fact can you tell me about Gary Cherone that you liked?\n        A:I like that Gary Cherone remained in contact and on good terms with Van Halen.\n        Q:What did Cherone do after Van Halen?\n        UQ:What did Gary Cherone do after Van Halen?\n        A:After his departure from Van Halen, Gary Cherone returned to Boston and put together a new project, Tribe of Judah.\n        Q:Did they release any albums during th

In [ ]:
lora_r = 64
lora_alpha = 16
lora_dropout = 0.1

use_4bit = True
bnb_4bit_compute_dtype = "float16"
bnb_4bit_quant_type = "nf4"
use_nested_quant = False

output_dir = "./results"
num_train_epochs = 1
fp16 = False
bf16 = False
per_device_train_batch_size = 4
per_device_eval_batch_size = 4
gradient_accumulation_steps = 1
gradient_checkpointing = True
max_grad_norm = 0.3
learning_rate = 2e-4
weight_decay = 0.001
optim = "paged_adamw_32bit"
lr_scheduler_type = "cosine"
max_steps = -1
warmup_ratio = 0.03
group_by_length = True
save_steps = 0
logging_steps = 25
max_seq_length = None
packing = False
device_map = {"": 0}

In [ ]:
from datasets import Dataset
dataset=Dataset.from_list(temp)
dataset

Dataset({
    features: ['text'],
    num_rows: 1000
})

In [ ]:
print(dataset['text'][10])

<s>
        Q:How did Van Halen reunite with Roth?
        UQ:How did Van Halen reunite with David Lee Roth?
        A:David Lee Roth called Eddie Van Halen to discuss what tracks would be included on a planned Van Halen compilation. Shortly afterwards, Roth re-entered the studio with the band.
        Q:Did Roth re-join the band?
        UQ:Did David Lee Roth re-join the band?
        A:Van Halen's appearance fueled reunion speculation. But several weeks after the awards show, it was discovered that David Lee Roth was out of Van Halen again.
        Q:Why was it only temporary?
        UQ:Why was David Lee Roth's renion only temporary?
        A:David Lee Roth released a statement in which he stated that he was an unwitting participant in a publicity stunt by manager Ray Danniels.
        Q:Did they release any songs during the time they were reunited?
        UQ:Did Van Halen release any songs during the time they were reunited?
        A:Two songs from those reunion sessions were ad

In [ ]:
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)


if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)


model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map=device_map
)
model.config.use_cache = False
model.config.pretraining_tp = 1


tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"
peft_config = LoraConfig(
    lora_alpha=lora_alpha,
    lora_dropout=lora_dropout,
    r=lora_r,
    bias="none",
    task_type="CAUSAL_LM",
)


config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

In [ ]:
# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=num_train_epochs,
    per_device_train_batch_size=per_device_train_batch_size,
    gradient_accumulation_steps=gradient_accumulation_steps,
    optim=optim,
    save_steps=save_steps,
    logging_steps=logging_steps,
    learning_rate=learning_rate,
    weight_decay=weight_decay,
    fp16=fp16,
    bf16=bf16,
    max_grad_norm=max_grad_norm,
    max_steps=max_steps,
    warmup_ratio=warmup_ratio,
    group_by_length=group_by_length,
    lr_scheduler_type=lr_scheduler_type,
    report_to="tensorboard"
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    max_seq_length=max_seq_length,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=packing,
)

# Train model
trainer.train()

# Save trained model
trainer.model.save_pretrained(new_model)

/opt/conda/lib/python3.10/site-packages/peft/utils/other.py:102: FutureWarning: prepare_model_for_int8_training is deprecated and will be removed in a future version. Use prepare_model_for_kbit_training instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/trl/trainer/sft_trainer.py:159: UserWarning: You didn't pass a `max_seq_length` argument to the SFTTrainer, this will default to 1024
  warnings.warn(


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/opt/conda/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torch/nn/parallel/_functions.py:68: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn('Was asked to gather along dimension 0, but all '


Step,Training Loss
25,1.450400
50,1.100300
75,1.074000
100,1.019400
125,0.973100


In [ ]:


pipe = pipeline(task="text-generation", model=model, tokenizer=tokenizer, max_length=300)
history= ""

while True:
        prompt = input(">>> ")
        while not prompt:
            print('Prompt should not be empty!')
            prompt = input(">>> ")

        if (len(history)==0):
          prompt1=f'''
        <s>
        <INST>Question:{prompt}</INST>
        Updated Question:'''

        else:
           prompt1=f'''\n<s>
           <INST>Quesion:{prompt}</INST>\nUpdated Question:'''

        result = pipe(history+prompt1)
        temp=result[0]['generated_text']
        temp=temp.replace('<s>',"")
        temp=temp.replace('</s>',"")
        temp=temp.replace('</s',"")
        temp=temp.replace('<s>\n',"")
        temp=temp.replace('</s\n>',"")
        temp=temp.replace('<INST>',"")
        temp=temp.replace('</INST>',"")
        temp=temp.replace('</question>\n','')
        temp=temp.replace('</answer>\n','')
        temp=temp.replace('Question','Q')
        temp=temp.replace('Updated Question','UQ')
        temp=temp.replace('Answer','A')
        temp=temp.replace('</Q>\n','')
        temp=temp.replace('</A>\n','')
        temp=temp.replace('</Q>','')
        temp=temp.replace('</Q','')
        temp=temp.replace('</A>','')
        temp=temp.replace('\n\n','\n')
        temp=temp.replace('</question',"")
        temp=temp.strip()
        print(temp)
        history=history+"\n"+temp



>>>  Who is President of USA?


Q:Who is President of USA?
        Updated Q:Who is the President of the United States?
        A:Joe Biden
</


>>>  When was he born?


Q:Who is President of USA?
        Updated Q:Who is the President of the United States?
        A:Joe Biden
</
           Quesion:When was he born?
Updated Q:When was Joe Biden born?
A:Joe Biden was born on November 20, 1942, in Scranton, Pennsylvania
</


>>>  What is his age?


Q:Who is President of USA?
        Updated Q:Who is the President of the United States?
        A:Joe Biden
</
Q:Who is President of USA?
        Updated Q:Who is the President of the United States?
        A:Joe Biden
</
           Quesion:When was he born?
Updated Q:When was Joe Biden born?
A:Joe Biden was born on November 20, 1942, in Scranton, Pennsylvania
</
           Quesion:What is his age?
Updated Q:What is Joe Biden's age?
A:Joe Biden is 80 years old
</
</








</Q
